In [2]:
import stim
import numpy as np
import random

## Data structure for Node 
this is used for:
- referencing to the qubits in the protocol
- measurement tree for each RGS arm
- 

## Implementation Details

Qubits indices:
- qubit 0 and 1 are located at end nodes for anchoring half-RGS, Alice and Bob respectively. So the goal is to verify the stabilizers of qubit 0 and 1 and expecting to see XZ and ZX.
- qubit 2 and 3 are for building the biclique RGS. The RGS is generated sequentially so we can reuse the qubits.



In [3]:
### helper functions

def num_qubits_per_rgs_arm(bvec: list[int]) -> int:
    num_in_layers = bvec[:]
    for i in range(1, len(num_in_layers)):
        num_in_layers[i] *= num_in_layers[i-1]
    num_qubits_per_arm = 1 + np.sum(num_in_layers)
    return num_qubits_per_arm

In [4]:
from enum import Enum

class Node:
    def __init__(self, qubit_index=-1, parent_index=-1):
        self.qubit_index = qubit_index  # for debugging
        self.parent_index = parent_index  # for debugging
        self.measurement_result: bool | None = (
            None  # this should be True and False if the qubit has been measured indicating the raw measurement result
        )
        self.eigenvalue: bool = False # use this to track the decoded measurement (after taking side effect and raw results into account)
        self.children: list[Node] = []
        self.is_lost = False  # this is used to denote whether the qubit is lost in the fiber or not
        self.has_z = False  # this is used to denote whether the qubit has Z side effect from the emission process or not (only from the emission!!)

    def get_post_order(self):
        return_list: list[Node] = []
        for u in self.children:
            return_list.extend(u.descendents_post_order())
        return [self] + return_list

    def get_level_traversal(self):
        return_list = [self]
        queue = [self]
        while len(queue) > 0:
            new_queue = []
            for u in queue:
                return_list.append(u)
                new_queue.extend(u.children)
            queue = new_queue
        return return_list

    def add_z_side_effects(self, circuit: stim.Circuit, z_prob=0.5):
        if len(self.children) == 0:
            return
        for u in self.children:
            u.add_z_side_effects(circuit, z_prob)
        if self.parent_index == -1:
            return
        if random.random() < z_prob:
            circuit.append("Z", self.qubit_index)
            self.has_z = not self.has_z

    def get_indices_from_level(self, k: int) -> list[int]:
        # Not that we need it right now?
        """get all the indices from the nodes"""
        cur_level = 0
        queue = [self]
        while len(queue) > 0:
            # arrive at the correct level, return the qubit indices
            if cur_level == k:
                return [v.qubit_index for v in queue]
            # need to go to the next level
            num_nodes_in_level = len(queue)
            for _ in range(num_nodes_in_level):
                u = queue.pop(0)
                queue.extend(u.children)
            cur_level += 1
        # should change this to an error
        return []  # empty list indicating the level specified is out of range

In [10]:
from enum import Enum

# functional syntax
Pauli = Enum('Basis', ['I', 'X', 'Y', 'Z'])

def helper_assign_qubit_indices(root: Node, bv: list[int], starting_index: int) -> int:
    cur_index = starting_index
    root.qubit_index = cur_index
    cur_index += 1
    queue = [root]
    for bi in bv:
        temp_queue = []
        for u in queue:
            for v in range(bi):
                v = Node(cur_index, u.qubit_index)
                cur_index += 1
                u.children.append(v)
                temp_queue.append(v)
        queue = temp_queue
    # u = self.arms[0]
    # for i in range(len(self.bv) + 1):
    #     vs = u.get_indices_from_level(i)
    #     print(f'{i} - {vs}')

    # return the next unused index
    return cur_index

def helper_initialize_rgs_arm(t: stim.TableauSimulator, root: Node, anchor: int, outer_emitter: int, root_ancilla: int) -> bool:
    # return whether the anchor should be flipped or not (side effects to the anchor)

    # generate outer qubit
    t.reset(outer_emitter, root_ancilla)
    t.h(root.qubit_index, outer_emitter)
    t.cz(root.qubit_index, outer_emitter)
    
    # generate inner qubit tree
    t.h(root_ancilla)
    queue = root.children # nodes in the first level
    for u in queue:
        t.h(u.qubit_index)
        t.cz(root_ancilla, u.qubit_index)
    # assuming that the anchor is already has Hadamard applied
    while len(queue) > 0:
        temp_queue = []
        for u in queue:
            for v in u.children:
                t.h(v.qubit_index)
                t.cz(u.qubit_index, v.qubit_index)
                temp_queue.append(v)
        queue = temp_queue
    
    # add random side effects to nodes in the tree except the leaves
    queue = root.children
    while len(queue) > 0:
        temp_queue = []
        for u in queue:
            if len(u.children) == 0:
                break
            if random.random() < 0.5:
                t.z(u.qubit_index)
                u.has_z = not u.has_z
            temp_queue.extend(u.children)
        queue = temp_queue

    # join inner and outer qubits
    t.cz(anchor, outer_emitter)
    t.cz(root_ancilla, outer_emitter)
    t.h(outer_emitter, root_ancilla)
    meas_outer = t.measure(outer_emitter)
    meas_root = t.measure(root_ancilla)
    if meas_outer:
        # flip first level qubits
        for u in root.children:
            u.has_z = not u.has_z

    if meas_root:
        # flip outer qubits (and return the flip to the anchor)
        root.has_z = not root.has_z

    return meas_root


def helper_process_photon_loss(t: stim.TableauSimulator, root: Node, loss_probability: float):
    """traverse the tree and apply loss probability to all qubits
    If a qubit is lost, randomly select Pauli X, Y, or Z to apply followed by a measurement in the Z basis.
    """
    queue = [root]
    while len(queue) > 0:
        temp_queue = []
        for u in queue:
            temp_queue.extend(u.children)
            if random.random() < loss_probability:
                q = u.qubit_index
                u.is_lost = True
                pauli_op = random.choice(["I", "X", "Y", "Z"])
                if pauli_op == "X":
                    t.x(q)
                elif pauli_op == "Y":
                    t.y(q)
                elif pauli_op == "Z":
                    t.z(q)
                t.measure(q)
        queue = temp_queue


def helper_update_side_effect_outer(outer: Node):
    if not outer.is_lost and outer.has_z:
        outer.eigenvalue = not outer.eigenvalue


def helper_update_side_effect_inner(root: Node, is_measured_x: bool):
    if not root.is_lost and is_measured_x and root.has_z:
        root.eigenvalue = not root.eigenvalue
    for v in root.children:
        helper_update_side_effect_inner(v, not is_measured_x)


class HalfRGS:
    """Currently this is only used at end nodes"""
    def __init__(self, m: int, branching_params: list[int], anchor_index: int):
        self.m = m
        self.bv = branching_params
        self.arms = [Node() for _ in range(m)]
        self.anchor = anchor_index
        self.measurement_bases: list[Pauli | None] = [None for _ in range(m)]
        self.successful_arm_index = -1

    def assign_qubit_indices(self, starting_index: int) -> int:
        """Assign qubits to half RGS
        Return: next unused qubit index"""
        cur_index = starting_index
        for root in self.arms:
            cur_index = helper_assign_qubit_indices(root, self.bv, cur_index)
        return cur_index

    def initialize_quantum_state(self, t: stim.TableauSimulator, outer_emitter: int, root_ancilla: int):
        anchor_has_z = False
        t.h(self.anchor)
        for root in self.arms:
            anchor_has_z = anchor_has_z ^ helper_initialize_rgs_arm(t, root, self.anchor, outer_emitter, root_ancilla)
        if anchor_has_z:
            t.z(self.anchor)

    def process_photon_loss(self, t: stim.TableauSimulator, loss_probability: float):
        for root in self.arms:
            helper_process_photon_loss(t, root, loss_probability)

    def update_measurement_with_side_effects(self, swap_index: int):
        """using the side effect stored in .has_z to update .eigenvalues"""
        # the root (outer) is always measured in X

        # if not root.is_lost and is_measured_in_x and root.has_z:
        #     root.eigenvalue = not root.eigenvalue
        # for v in root.children:
        #     update_measurement_with_side_effects(v, not is_measured_in_x)
        pass


class RGS:
    def __init__(self, m: int, branching_params: list[int]):
        self.m = m
        self.bv = branching_params
        self.left_arms = [Node() for _ in range(m)]
        self.right_arms = [Node() for _ in range(m)]
        self.measurement_bases_left: list[Pauli | None] = [None for _ in range(m)]
        self.measurement_bases_right: list[Pauli | None] = [None for _ in range(m)]

    def assign_qubit_indices(self, starting_index: int) -> int:
        """Assign qubits to be used for the photonic qubits of the RGS
        Return: next unused qubit index"""
        cur_index = starting_index
        for root in self.left_arms:
            cur_index = helper_assign_qubit_indices(root, self.bv, cur_index)
        for root in self.right_arms:
            cur_index = helper_assign_qubit_indices(root, self.bv, cur_index)
        return cur_index

    def initialize_quantum_state(self, t: stim.TableauSimulator, anchor_left: int, anchor_right: int, outer_emitter: int, root_ancilla: int):
        # make sure the qubits are properly initialized
        t.reset(anchor_left, anchor_right)

        # generate the left arms
        anchor_has_z = False
        t.h(anchor_left)
        for root in self.left_arms:
            anchor_has_z = anchor_has_z ^ helper_initialize_rgs_arm(t, root, anchor_left, outer_emitter, root_ancilla)
        if anchor_has_z:
            # we fix the anchor as should be done by the RGSS during the generation process
            t.z(anchor_left)

        # generate the right arms
        anchor_has_z = False
        t.h(anchor_right)
        for root in self.left_arms:
            anchor_has_z = anchor_has_z ^ helper_initialize_rgs_arm(t, root, anchor_right, outer_emitter, root_ancilla)
        if anchor_has_z:
            t.z(anchor_right)

        # join the two halves
        t.cz(anchor_left, anchor_right)
        t.h(anchor_left, anchor_right)
        meas_left = t.measure(anchor_left)
        meas_right = t.measure(anchor_right)

        # tracking the side effects (toggling first level nodes of all arms)
        if meas_left:
            for root in self.right_arms:
                for u in root.children:
                    u.has_z = not u.has_z
        if meas_right:
            for root in self.left_arms:
                for u in root.children:
                    u.has_z = not u.has_z

    def process_photon_loss(self, t: stim.TableauSimulator, loss_probability: float):
        for root in self.left_arms:
            helper_process_photon_loss(t, root, loss_probability)
        for root in self.right_arms:
            helper_process_photon_loss(t, root, loss_probability)

    def update_measurements_with_side_effect(self, left_index: int, right_index: int):
        pass


In [ ]:
def physical_measure_pattern(t: stim.TableauSimulator, root: Node, is_starting_with_x: bool):
    queue = root.children
    is_x_basis = is_starting_with_x
    while len(queue) > 0:
        temp_queue = []
        for u in queue:
            if is_x_basis:
                t.h(u.qubit_index)
            if not u.is_lost:
                # if the qubit is lost, don't record or assign measurement result
                u.eigenvalue = u.measurement_result = t.measure(u.qubit_index)
            temp_queue.extend(u.children)
        is_x_basis = not is_x_basis
        queue = temp_queue


def in_tree_measure_z(root: Node) -> bool | None:
    # TODO: implement majority vote mechanism
    # direct result available
    if not root.is_lost and root.measurement_result is not None:
        return root.measurement_result
    # indirect measurement
    for u in root.children:
        if u.measurement_result is None:
            continue
        next_level_zs = [in_tree_measure_z(v) for v in u.children]
        if not all([z is not None for z in next_level_zs]):
            continue
        if u.has_z:
            next_level_zs.append(not u.measurement_result)
        else:
            next_level_zs.append(u.measurement_result)
        return next_level_zs.count(True) % 2 != 0
    # measurement result cannot be determined neither direct nor indirect
    return None


def tree_logical_measure_z(root: Node) -> bool | None:
    """find the XOR (parity) of the first level nodes"""
    pass


def tree_logical_measure_x(root: Node) -> bool | None:
    """find successful X in the first level and return the parity of it with all its children's Z results
    i.e., the parity of the X_i Z_children_of_i of any first level node i"""
    # TODO: implement majority vote mechanism
    pass


def measurements_at_absa(
    t: stim.TableauSimulator, m: int, left_halfs: list[Node], right_halfs: list[Node]
) -> int:
    """measurement of all qubits in the RGS (step 1) and return the index of the arm that has a successful BSM"""
    if m != len(left_halfs) or m != len(right_halfs):
        ValueError(f'number of arms {m} does not equal the input length of list of two halves {len(left_halfs)}, {len(right_halfs)}')

    success_arm_index = -1
    for i in range(m):
        unode = left_halfs[i]
        vnode = right_halfs[i]
        # check lost
        if unode.is_lost or vnode.is_lost:
            continue

        u = unode.qubit_index
        v = vnode.qubit_index
        t.cz(u, v)
        t.h(u, v)
        unode.measurement_result = unode.eigenvalue = t.measure(u)
        vnode.measurement_result = vnode.eigenvalue = t.measure(v)

        # mark successful arm index and physical measurements of all inner qubits
        if success_arm_index != -1 and unode.measurement_result != vnode.measurement_result:
            success_arm_index = i
            physical_measure_pattern(t, unode, True)
            physical_measure_pattern(t, vnode, True)
        else:
            physical_measure_pattern(t, unode, False)
            physical_measure_pattern(t, vnode, False)

    return success_arm_index

def update_tree_with_outer_qubits(left_tree_root: Node, right_tree_root: Node):
    """update in place with the BSM results; toggling 1st level results with root of another tree (step 2)"""
    # error checking
    if left_tree_root.is_lost or right_tree_root.is_lost:
        RuntimeError("trying to update tree with outer qubits that were lost!")
    if left_tree_root.eigenvalue:
        for u in right_tree_root.children:
            if u.is_lost: 
                continue
            u.eigenvalue = not u.eigenvalue
    if right_tree_root.eigenvalue:
        for u in left_tree_root.children:
            if u.is_lost: 
                continue
            u.eigenvalue = not u.eigenvalue

def compute_parity_for_end_nodes(m: int, left_halfs: list[Node], right_halfs: list[Node], successful_bsm_index: int) -> tuple[bool, bool]:
    """apply the parity at end nodes (step 3)
    return tuple of parity to be sent to the left and right respectively"""
    # parity multiplied together of Z of left (right) is sent to right (left),
    # while X of left (right) is sent to left (right).
    left_par, right_par = False, False
    for i in range(m):
        if i == successful_bsm_index:
            continue
        left_par ^= left_halfs[i].eigenvalue
        right_par ^= right_halfs[i].eigenvalue
    left_par ^= right_halfs[successful_bsm_index].eigenvalue
    right_par ^= left_halfs[successful_bsm_index].eigenvalue
    return left_par, right_par

In [1]:
def experiment_setup(
    number_of_hops: int,
    m: int,
    branching_parameters: list[int],
    loss_probability: float = 0,
    # photon_error_probability: float = 0,
    # emitter_error_probability: float = 0,
):
    """In this method, we assign the qubit index to all the objects we will create and manage"""
    num_of_qubits_in_half_rgs = num_qubits_per_rgs_arm(branching_parameters)
    # Ancilla qubits we require (total 4)
    #   temporary anchor for tree encoding: 1 (ancilla[0])
    #   emitter for outer qubit: 1 (ancilla[1])
    #   anchor for the half-RGS: 2 (ancilla[2-3])
    # qubit range
    #   Alice:      0, 4 to (num_of_qubits_in_half_rgs + 3)
    #   Bob:        1, (num_of_qubits_in_half_rgs + 4) to (2 * num_of_qubits_in_half_rgs + 3)
    #   first RGS:  (2 * num_of_qubits_in_half_rgs + 4) to (3 * num_of_qubits_in_half_rgs + 3)
    #   ...
    alice = 0
    bob = 1
    # just for accounting at the moment
    anchor_left = 2
    anchor_right = 3
    outer_emitter = 4
    root_id = 5
    next_id = 6

    # we need (hop - 1) RGS
    rgss = [RGS(m, branching_parameters) for _ in range(number_of_hops - 1)]
    half_alice = HalfRGS(m, branching_parameters, alice)
    half_bob = HalfRGS(m, branching_parameters, bob)

    # assign qubit indices
    for rgs in rgss:
        next_id = rgs.assign_qubit_indices(next_id)
    next_id = half_alice.assign_qubit_indices(next_id)
    next_id = half_bob.assign_qubit_indices(next_id)

    # step 1: RGS creation
    t = stim.TableauSimulator()
    for rgs in rgss:
        rgs.initialize_quantum_state(t, anchor_left, anchor_right, outer_emitter, root_id)
    half_alice.initialize_quantum_state(t, outer_emitter, root_id)
    half_bob.initialize_quantum_state(t, outer_emitter, root_id)

    # step 2: process photon loss
    for rgs in rgss:
        rgs.process_photon_loss(t, loss_probability)
    half_alice.process_photon_loss(t, loss_probability)
    half_bob.process_photon_loss(t, loss_probability)

    # step 3: ABSA measurements
    #         first with all the rgss, then at the end nodes
    success_bsm_indices = [-1] * number_of_hops
    for i in range(len(rgss) - 1):
        success_bsm_indices[i+1] = measurements_at_absa(t, m, rgss[i].right_arms, rgss[i+1].left_arms)
    if len(rgss) > 0:
        success_bsm_indices[0] = measurements_at_absa(t, m, half_alice.arms, rgss[0].left_arms)
        success_bsm_indices[-1] = measurements_at_absa(t, m, rgss[-1].right_arms, half_bob.arms)
    else:
        # special case for 1 hop (no RGSS source nodes)
        success_bsm_indices[0] = measurements_at_absa(t, m, half_alice.arms, half_bob.arms)

    if any(map(lambda id: id == -1, success_bsm_indices)):
        # fail trial
        pass

    # step 4: Update measurements tree (assigning eigenvalues taking the side effects into account from the physical measurement results)
    half_alice.update_measurement_with_side_effects(success_bsm_indices[0])
    half_bob.update_measurement_with_side_effects(success_bsm_indices[-1])
    for i, rgs in enumerate(rgss):
        rgs.update_measurements_with_side_effect(success_bsm_indices[i], success_bsm_indices[i+1])

    # step 5: propagating side effects of BSMs into connected inner qubits
    bsm_arm_pairs = [half_alice.arms[success_bsm_indices[0]]]
    for i in range(len(rgss)):
        bsm_arm_pairs.append(rgss[i].left_arms[success_bsm_indices[i]])
        bsm_arm_pairs.append(rgss[i].right_arms[success_bsm_indices[i+1]])
    bsm_arm_pairs.append(half_bob.arms[success_bsm_indices[-1]])
    for i in range(0, len(bsm_arm_pairs), 2):
        update_tree_with_outer_qubits(bsm_arm_pairs[i], bsm_arm_pairs[i+1])


    # step 6: compute parity for memory at end nodes
    parities: list[tuple[bool, bool]] = []
    if number_of_hops == 1:
        parities.append(compute_parity_for_end_nodes(m, half_alice.arms, half_bob.arms, success_bsm_indices[0]))
    else:
        parities.append(compute_parity_for_end_nodes(m, half_alice.arms, rgss[0].left_arms, success_bsm_indices[0]))
        for i in range(len(rgss) - 1):
            parities.append(compute_parity_for_end_nodes(m, rgss[i].right_arms, rgss[i+1].left_arms, success_bsm_indices[i+1]))
        parities.append(compute_parity_for_end_nodes(m, rgss[-1].right_arms, half_bob.arms, success_bsm_indices[-1]))
    total_parity = (False, False)
    total_parities = [total_parity := (total_parity[0] ^ p[0], total_parity[1] ^ p[1]) for p in parities]
    if total_parity[0]:
        t.z(alice)
    if total_parity[1]:
        t.z(bob)

    # verifying that we actually have bipartite graph state (ZX, XZ) stabilizers
    print(t.peek_observable_expectation(stim.PauliString('ZX')), t.peek_observable_expectation(stim.PauliString('XZ')))

In [1]:
def experiment_run():
    # setup
    # RGS creation
    # ABSA measurements
    
    """
    steps to perform one trial
    0. experiment_setup
    1. RGS creation --> store side effect (has Z)
    2. process photon loss
    3. ABSA measurements (create measurement trees) --> store the measurement value (raw result)
        a. BSM
        b. Single qubit measurements
    4. Protocol (according to Sec 6 or 7?)
        a. measurement tree updated with the side effect (this is actually done prior at RGS creation) --> store the eigenvalue (from raw result and side effect)
        b. update from BSM (the pair tree) --> 
        c. parity multiplied together of Z of left (right) is sent to right (left), 
            while X of left (right) is sent to left (right)
    """
    